#Demo de TF-Agents para resolver el problema de encontrar un Máximo Óptimo definido al azar en un plano haciendo mover un Agente de tipo Hormiga-Tortuga (ANT-Turtle):


0) Preparar el ambiente:

In [ ]:
#@title Instalar Paquete de TF-Agents
!pip install -q tf-agents
print("TF-Agentes instalado.")

In [ ]:
#@title Cargar Librerías
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import abc
import tensorflow as tf
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import random
from random import randint, sample
import math
import operator
import copy
import pickle
import codecs
import IPython


from tf_agents.environments import py_environment
from tf_agents.environments import tf_py_environment

from tf_agents.environments import utils
from tf_agents.specs import array_spec

from tf_agents.policies import random_tf_policy

from tf_agents.trajectories import time_step as ts

from tf_agents.agents.dqn import dqn_agent
from tf_agents.networks import q_network
from tf_agents.agents.categorical_dqn import categorical_dqn_agent
from tf_agents.networks import categorical_q_network

from tf_agents.utils import common

from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.trajectories import trajectory

import os
from tf_agents.policies import policy_saver

tf.compat.v1.enable_v2_behavior()

print("Librerías cargadas.")

1) Establecer las clases sobre el Problema a resolver:

In [ ]:
#@title Parámetros Generales:
# Espacio mínimo y máximo del espacio de búsqueda
CANT_MAXIMOS_LOCALES = 1 #@param {type:"slider", min:0, max:10, step:1}
MIN_ESPACIO_BUSQ = 0 #@param {type:"slider", min:-100, max:0, step:5}
MAX_ESPACIO_BUSQ = 30  #@param {type:"slider", min:0, max:100, step:5}
CANT_OBSTACULOS = 100  #@param {type:"slider", min:0, max:1000, step:10}

# otros parámetros fijos
EVAPORACION_FEROMONAS = 1
ATRACCION_FEROMONAS = 0
ATRACCION_HEURISTICA = 0

print("Parámetros definidos.")

In [ ]:
#@title Definir clase MAPA DE BÚSQUEDA (notar que es la misma usada en ANT de SWARM)
# Nota: las funciones de uso de feromonas se mantienen por compatibilidad pero no se usan

class MapaBusqueda(object):

    def __init__(self, limMin, limMax, cantMaximosLocales=0, cantObstaculos=0, valorIniFeromonas=1):
          self.limMin = limMin
          self.limMax = limMax
          self.cantMax = cantMaximosLocales
          self.cantObstaculos = cantObstaculos
          self.generarPosicionesRnd()
          self.inicializarFeromonas()

    def _reset(self):
          #self.generarPosicionesRnd()
          self.inicializarFeromonas()

    def generarPosicionesRnd(self):
      
        # Define Posiciíon inicial de las Hormigas
        self.Hormiguero = [ random.randint(self.limMin, self.limMax),  random.randint(self.limMin, self.limMax) ]      

        # Define Posiciones Random para Máximos Locales y Óptimos
        # con Heurística también al Azar (entre 1 y 100)
        # controla que el máximo local/global no se ubique en la misma posición que el Hormiguero
        self.MaximosPos = []
        self.MaximosVal = []
        self.MaxOptimoID = 0
        auxMejorVal = -99
        i = 0

        # Agrega el maximo global (siempre vale 100)
        auxPos = [ random.randint(self.limMin, self.limMax),  random.randint(self.limMin, self.limMax) ]
        self.MaximosPos.append( auxPos )
        self.MaximosVal.append( 100 )  

        # Agrega máximos locales
        while len(self.MaximosPos)<(self.cantMax):

            auxPos = [ random.randint(self.limMin, self.limMax),  random.randint(self.limMin, self.limMax) ]      

            if (self.Hormiguero != auxPos):
                auxVal = random.randint(1, 90)
                self.MaximosPos.append( auxPos )
                self.MaximosVal.append( auxVal )
                
                if self.MaximosVal[self.MaxOptimoID] < auxVal:
                    self.MaxOptimoID = i
                i= i + 1

        # Agrega los obstáculos controlando que no se ubique en una posición igual al hormiguero o a un máximo
        self.ObstaculosPos = []
        while len(self.ObstaculosPos)<self.cantObstaculos:
              auxPos = [ random.randint(self.limMin, self.limMax),  random.randint(self.limMin, self.limMax) ]
                  
              i = 0
              agregaObstaculo = True
              while agregaObstaculo and i<len(self.MaximosPos):
                if (self.MaximosPos[i] == auxPos) or (self.Hormiguero == auxPos):
                      agregaObstaculo = False                      
                i= i + 1
              
              if agregaObstaculo:
                    self.ObstaculosPos.append( auxPos )      

        return True
   
    def printInformacion(self):
        # Muestra la información sobre el Mapa de Búsqueda
        print("\n++ Ubicación del Hormiguero: ", self.Hormiguero)

        print("\n## Obstáculos Generados: ", len(self.ObstaculosPos))
        for auxPos in self.ObstaculosPos:
            print("  -->", auxPos)

        print("\n** Máximos Generados: ", len(self.MaximosPos))
        for auxPos, auxVal in zip(self.MaximosPos, self.MaximosVal):
            print("  -->", auxPos, "{ ", auxVal, " } ")
        print("\n** Posición Máximo Óptimo: ", self.MaximosPos[self.MaxOptimoID], " { ",  self.MaximosVal[self.MaxOptimoID]," } **")
        print("\n")

    @property
    def posHormiguero(self):
        return self.Hormiguero         

    @property
    def posMaximos(self):
        return self.MaximosPos    

    @property
    def valMaximos(self):
        return self.MaximosVal   
        
    @property
    def posMaximoGlobal(self):
        return self.MaximosPos[self.MaxOptimoID]         
    
    @property
    def valMaximoGlobal(self):
        return self.MaximosVal[self.MaxOptimoID]  

    @property
    def posObstaculos(self):
        return self.ObstaculosPos

    def estaEnMaximo(self, posicion):
      # Indica si la posición corresponde o no a un Máximo 
      # si es verdadero devuelve el valor correspondiente, sino 0
      for i in range(len(self.MaximosPos)):
        if posicion == self.MaximosPos[i]:
          return self.MaximosVal[i]
      return 0

    def heuristica(self, posicion):
        # Define la Función Heurística para evaluar una posicion del mapa
        devuelve = 0
        disPosMenor = self.limMax
        for posMax, valMax in zip(self.MaximosPos, self.MaximosVal):

              # calcula la distancia al punto de la Particula
              sqerrors = ((x - y)**2 for x, y in zip(posicion, posMax))
              distPos =  math.fsum(sqerrors)**0.5 / 10
              
              # Si es la menor distancia calcula la Heurística
              if disPosMenor > distPos:
                  disPosMenor = distPos 
                  devuelve = ( valMax - distPos) 

          # Por las dudas controla que devuelva un valor positivo
        return max(devuelve,0)

    def inicializarFeromonas(self):
        # Inicializa el mapa de feromonas como un diccionario del tamaño prefijado con valor constante 
        self.mapaFeromonas = dict()
        self.feroDft = 1
        ## Nota: no se inicializa el mapa de feromonas con valores para no llenar la memoria
        ## lo que se hace es que si una posición es solicitada y no existe, 
        ## se asume que el valor de 'self.feroDft'
        #for x in range(self.limMin,self.limMax):
        #    for y in range(self.limMin,self.limMax):              
        #      self.mapaFeromonas[x,y] = 1
        return True

    def evaporarFeromonas(self, factorEvaporacion=EVAPORACION_FEROMONAS):
        # actualiza el valor por defecto
        self.feroDft = (1-factorEvaporacion)*self.feroDft

        # actualiza el valor de las posiciones definidas
        for pos in self.mapaFeromonas:
            self.mapaFeromonas[pos] = (1-factorEvaporacion)*self.mapaFeromonas[pos] 
        return True

    def actualizaFeromonasRecorrido(self, recorrido, valorRecorrido):
        # si el valor del recorrido es cero o menos, no se actualiza la cantidad de feromonas
        if valorRecorrido<=0:
          return False
        # si es mayor a cero se actualizan las feromonas 
        # de acuerdo al recorrido que tuvieron la hormigas y su valor
        valAplicar = valorRecorrido/100
        for posRec in recorrido:
              tpos = tuple(posRec)
              if tpos in self.mapaFeromonas:
                    self.mapaFeromonas[tpos] = self.mapaFeromonas[tpos] + valAplicar
              else:
                    self.mapaFeromonas[tpos] = self.feroDft + valAplicar
        return True

    def cantFeromonas(self, posicion):
        # Nota: como no se inicializa el mapa de feromonas con valores para no llenar la memoria
        # lo que se hace es que si una posición es solicitada y no existe, 
        ## se asume que el valor de 'self.feroDft' 
        if (self.mapaFeromonas == None) or (self.mapaFeromonas == dict()):
            return self.feroDft
        tpos = tuple(posicion)
        if tpos in self.mapaFeromonas:
            return self.mapaFeromonas[tpos]
        else:
            return self.feroDft

    def mostrarMapaFeromonas(self):
        # Genera el gráfico con el Mapa de Fermonas:
        plt.figure(figsize=(15,8)) 

        ## -- en Amarillo: para feromonas menores a 1 
        ## -- en Naranja: para feromonas iguales  a 1 
        ## -- en Marron: para feromonas mayores a 1 
        if self.feroDft < 1:   
            plt.rcParams['axes.facecolor'] = 'yellow'
        elif self.feroDft == 1:   
            plt.rcParams['axes.facecolor'] = 'orangered'
        else:
            plt.rcParams['axes.facecolor'] = 'saddlebrown'  
        for pos in self.mapaFeromonas:
            if self.mapaFeromonas[pos] < 1:
                plt.scatter(pos[0], pos[1], color='yellow')
            elif self.mapaFeromonas[pos] == 1:
                plt.scatter(pos[0], pos[1], color='orangered')
            else:
                plt.scatter(pos[0], pos[1], color='saddlebrown')
        
        ## -- en Verde: Posición del Máximo Global 
        plt.scatter(mapa.posMaximoGlobal[0], mapa.posMaximoGlobal[1], color='green', s=150)

        ## --en Violeta: Posición del Homiguero
        plt.scatter(mapa.posHormiguero[0], mapa.posHormiguero[1], color='violet', s=100)

        plt.title('Mapa de Feromonas')        
        plt.xlim(self.limMin, self.limMax)
        plt.ylim(self.limMin, self.limMax)
        plt.grid(False)
        plt.show()
      
    def probabAsignada(self, posicion, atraccFeromonas=ATRACCION_FEROMONAS, atraccHeuristica=ATRACCION_HEURISTICA):           
        # si es una ubicación de un obstáculo, devuelve 0 para que no sea elegida
        for o in self.ObstaculosPos:
            if o == posicion:
              return 0
        
        # si la posición está fuera del espacio de búsqueda, devuelve 0 para que no sea elegida
        for pos in posicion:                
            if pos<self.limMin:
                  return 0
            if pos>self.limMax:
                  return 0

        # determina el valor de probabilidad correspondiente        
        f = self.cantFeromonas(posicion)
        h = self.heuristica(posicion)
        probab = f**atraccFeromonas * h**atraccHeuristica

        return probab   

print("\nClase Mapa de Búqueda definida") 


In [ ]:
#@title Definir clase Ant-Turtle (basada en la usada para SWARM pero con cambios)
# Nota: en esta versión no se usan las feromonas y se agregan métodos para desplazar más acotado

class HormigaTortuga(object):

    def __init__(self, posInicial):        
        self.posActual = posInicial
        self.despRecorrido = [ [posInicial] ]
        self.direccionGrados = 0
        self.valRecorrido = 0

    @property
    def recorridoEncuentraSolucion(self):
      return (self.valRecorrido > 0)

    @property
    def recorridoUltimo(self):
        return self.despRecorrido   

    @property
    def valRecorridoUltimo(self):
        return self.valRecorrido

    @property
    def direccionActual(self):
        return self.direccionGrados

    @property
    def posicionActual(self):
        return self.posActual

    def determinaMovDireccion(self, direccionGrados):
        # determina hacia donde se mueve usando los grados de la dirección
        if direccionGrados >= 0 and direccionGrados < 45:
          mov = [0, -1]
        elif direccionGrados >= 45 and direccionGrados < 90:
          mov = [1, -1]
        elif direccionGrados >= 90 and direccionGrados < 135:
          mov = [1, 0]
        elif direccionGrados >= 135 and direccionGrados < 180:
          mov = [1, 1]
        elif direccionGrados >= 180 and direccionGrados < 225:
          mov = [0, 1]
        elif direccionGrados >= 225 and direccionGrados < 270:
          mov = [-1, 1]
        elif direccionGrados >= 270 and direccionGrados < 315:
          mov = [-1, 0]
        elif direccionGrados >= 315 and direccionGrados < 360:
          mov = [-1, -1]
        else:
          mov = [0, 0]
        return mov

    def evaluarPosicion(self, mapa, pos):
        # Se fija que no supere ningún límite
        if pos[0] < mapa.limMin or pos[0] > mapa.limMax:
          return -1
        elif pos[1] < mapa.limMin or pos[1] > mapa.limMax:
          return -1
        # Se fija que no sea un obstáculo      
        elif pos in mapa.posObstaculos:
          return -1
        else:
          # Evalúa si se encontro algún Máximo (local o global)
          return mapa.estaEnMaximo(pos)

    def ve(self, mapa, cantPasosVe=3):
        # devuelve matriz de objetos que ve
        # en la dirección que está y sus costados
        resVeCompleto = []       
        for iGrados in range(8):
          # inicializa lista auxiliar 
          # de lo que ve en esa dirección
          resVe = np.zeros(cantPasosVe)
          # inicializa dirección en que ve
          nDir = (self.direccionGrados + (iGrados*45)) % 360           
          if nDir in [270, 315, 0, 45, 90]:
            # si corresponde a adelante o los costados
            movVe = self.determinaMovDireccion(nDir)
            # inicializ posición inicial a ver
            posVe = copy.deepcopy( self.posActual )
            for i in range( cantPasosVe ):
              # determina nueva posición
              nPosVe = []
              for p, m in zip(posVe, movVe):
                nPosVe.append( p + m )            
              # se fija que hay en esa posición y lo registra
              resVe[i]  = self.evaluarPosicion(mapa, nPosVe)
              # actualiza posición
              posVe = nPosVe
          # registra en visión completa
          resVeCompleto.append( resVe )
        # devuelve 8 listas:
        #       5 listas (adelante y costados) con valores:
        #         -1 para obstáculos o limites
        #          0 para posiciones normales
        #         >0 para máximos
        #       3 listas (atrás) con valores en 0
        return resVeCompleto

    def recuerda(self, mapa, cantPasosRecuerda=3):
        # devuelve matriz de objetos que recuerda
        # en todas las direcciones
        todos_pos_recorrido = [pos for subRecorrido in self.despRecorrido for pos in subRecorrido]
        resRecuerdaCompleto = []    
        for iGrados in range(8):
          # inicializa dirección en que recuerda
          nDir = (self.direccionGrados + (iGrados*45)) % 360
          movRec = self.determinaMovDireccion(nDir)
          # inicializa posición inicial a recordar
          posRec = copy.deepcopy( self.posActual )
          # inicializa lista auxiliar 
          # de lo que recuerda en esa dirección
          resRec = np.ones(cantPasosRecuerda)
          for i in range( cantPasosRecuerda ):
            # determina nueva posición
            nPosRec = []
            for p, m in zip(posRec, movRec):
              nPosRec.append( p + m )            
            # se fija si paso por esa posición
            if nPosRec in todos_pos_recorrido:
              resRec[i] = 0
            # actualiza posición
            posRec = nPosRec
          # registra en visión completa
          resRecuerdaCompleto.append( resRec )
        # devuelve 8 listas con valores:
        #           1 para posiciones que no paso    
        #           0 para posiciones por las que paso          
        return resRecuerdaCompleto


    def desplazarse(self, mapa, gradosGirar, cantPasos=1):
        # contra que la cantidad de pasos sea válida
        if cantPasos <= 0:
          self.despRecorrido.append( [self.posActual] )
          return 0

        # calcula nuevos grados
        self.direccionGrados = (self.direccionGrados + gradosGirar) % 360      
        mov = self.determinaMovDireccion( self.direccionGrados )
        # si no es una dirección válida, no avanza
        if mov == [0, 0]:          
          self.despRecorrido.append( [self.posActual] )
          return 0         

        # realiza el desplazamiento
        paso = 1
        auxRecorrido = []
        while paso <= cantPasos:
          # simula nueva posición
          nPos = []
          for p, m in zip(self.posActual, mov):
            nPos.append( p + m )
          valNPos = self.evaluarPosicion(mapa, nPos)
          if valNPos == -1:
            # es un obstáculo por lo que no se puede seguir avanzando 
            # en esa dirección
            break
          else:
            # actualiza la posición actual
            self.posActual = nPos
            auxRecorrido.append( nPos )
            if valNPos > 0:
              # se encontro algún máximo, finaliza
              self.maxEncontrado = True
              self.valRecorrido = valNPos
              break
            else:
              # continua la búsqueda
              paso = paso + 1
        # devuelve la cantidad de pasos realizados
        self.despRecorrido.append( auxRecorrido )
        return (paso/cantPasos)

print("\nClase Ant-Turtle definida")


In [ ]:
#@title Definir funciones para generar el Gráfico con las posiciones de las partículas

# Librerías especiales para usar
import matplotlib as mpl
from matplotlib import animation, rc
from IPython.display import HTML

# método que se usa para generar gradiente de colores
def colorFader(c1, c2, mix=0.0): #fade (linear interpolate) from color c1 (at mix=0) to c2 (mix=1)
    c1 = np.array(mpl.colors.to_rgb(c1))
    c2 = np.array(mpl.colors.to_rgb(c2))
    return mpl.colors.to_hex((1-mix)*c1 + mix*c2)

# Método que se ejecuta por cada frame para mostrar 
def updatePlot(i, ciclosPos, scat, axi):
    if axi is None or ciclosPos is None:
      return scat,

    axi.set_xlabel('Ciclo: ' + str(i), fontsize=12)
  
      # Si no es el último ciclo, muestra también el global
    if i<len(ciclosPos):     

      if len(ciclosPos[i])>0:      
        # marca posiciones anteriores en tamaño más pequeño
        # Separa las coordenadas x, y de las posiciones en el ciclo i              
        X, Y = zip(*ciclosPos[i])
        axi.scatter(X, Y, color='red', s=10)
        
        # Hace mover a las Partículas (Rojo)
        scat.set_offsets(ciclosPos[i])   

    # Devuelve una lista de "artistas" para dibujar, 
    # en este caso es sólo uno por lo que se pone una coma final
    return scat,    

# Función para preparar el gráfico          
def PrepararGrafico(mapa, MIN_ESPACIO_BUSQ, MAX_ESPACIO_BUSQ, ciclosPos): 

    #fig = plt.figure(figsize=(14,7))
    fig = plt.figure(figsize=(13,6))
    ax = fig.add_subplot(111)
    plt.close()

    textoTitulo = "Gráfico del Movimiento de la Hormiga-Tortuga"
    textoDesc = '\n** Posición Máximo Óptimo a buscar : ' + str(mapa.posMaximoGlobal) + ' { ' +  str(mapa.valMaximoGlobal) +' } **'
    textoDesc = textoDesc + '\n-- Colores:  Máximos Locales (AZUL) - Máximo Óptimo (VERDE) - '  
    textoDesc = textoDesc + '\n- Obstáculos (NEGRO) - Hormiguero (VIOLETA) -- ' 
    textoDesc = textoDesc + '\n- Recorrido (ROJO) --- ' 

    ## --en Negro: posiciones de los obstáculos a esquivar
    for posi in mapa.posObstaculos:
        ax.scatter(posi[0], posi[1], color='black')

    ## -- en Azul: Máximos Locales (positivos)
    ## -- en Cyan: Mínimos Locales (negativos)
    ## -- en Verde: Máximo Óptimo
    for posi, vali in zip(mapa.posMaximos, mapa.valMaximos):    
      valRel = abs(vali / mapa.valMaximoGlobal) 
      if vali < 0:        
          c = colorFader('cyan', 'green', valRel) 
          ax.scatter(posi[0], posi[1], color=c)
      else:
          c = colorFader('blue', 'green', valRel) 
          ax.scatter(posi[0], posi[1], color=c)     
    ax.scatter(mapa.posMaximoGlobal[0], mapa.posMaximoGlobal[1], color='green', s=150)    

    ## --en Violeta: Posición del Homiguero
    ax.scatter(mapa.posHormiguero[0], mapa.posHormiguero[1], color='violet', s=100)

    # Define el tamaño de la figura
    ax.axis([MIN_ESPACIO_BUSQ, MAX_ESPACIO_BUSQ, MIN_ESPACIO_BUSQ, MAX_ESPACIO_BUSQ])

    # Separa las coordenadas x, y de las posiciones en el ciclo inicial
    X, Y = zip(*ciclosPos[0])
    ## --en Rojo: posiciones de las posiciones de cada ciclo
    scat = ax.scatter(X, Y, color='red', s=50)

    # Muestra título y texto debajo
    ax.set_title(textoTitulo)
    ax.set_ylabel(textoDesc, fontsize=11)

    # Luego setea la animación usando los dos métodos anteriores 
    cant = len(ciclosPos)
    ani = animation.FuncAnimation(fig, updatePlot, 
                                  frames=cant, interval=cant,
                                  fargs=(ciclosPos, scat, ax), 
                                  blit=True, repeat=False)
    
    return ani


# Define la configuración para el gráfico
MOSTRAR_HISTORICO_MOVIMIENTO = False

### Nota: esto se agega para que funcione en Google Colab
##rc('animation', html='jshtml')
##ani

print("Funciones para generar gráfico definidas.")

In [ ]:
#@title Definir clase del Entorno del Problema 

# parámetros generales para el entorno
# cantidad de pixeles en cada dirección que ve o recuerda 
OBS_MAX_PIXELES_VE_o_RECUERDA = 10 #@param {type:"integer"} 
if OBS_MAX_PIXELES_VE_o_RECUERDA <= 1:
  OBS_MAX_PIXELES_VE_o_RECUERDA = 3
OBS_MAX_DIRECCIONES_VE_o_RECUERDA = 8 # corresponde a las 8 direcciones que ve / recuerda
POSIBLES_ACCIONES_DESC = [ "mover" ]
CANT_MAX_PASOS_REALIZAR = OBS_MAX_PIXELES_VE_o_RECUERDA
MAXIMO_VALOR_ACTION = CANT_MAX_PASOS_REALIZAR*10

def parsearAccion(action):
  # id de tipo de acción siempre el mismo
  idAccion = 0
  # determina parámetros 
  # cantidad de pasos y grados de dirección
  ppasos = action//10
  aux  = action - ppasos*10
  pgrados = (aux * 45) % 360
  return idAccion, pgrados, ppasos

# Un entorno que represente el juego podría verse así:
class AntTurtleBuscarMaxMapaEntorno(py_environment.PyEnvironment):

  def __init__(self, cantMaxIteraciones=50, resetCambiaTodo=True, Hash_configuracion_Mapa_Busqueda=None):
    self._action_spec = array_spec.BoundedArraySpec(
        shape=(), dtype=np.int32, minimum=0, maximum=MAXIMO_VALOR_ACTION, name='action')
    self._observation_spec = array_spec.BoundedArraySpec(
        shape=(2, OBS_MAX_DIRECCIONES_VE_o_RECUERDA, OBS_MAX_PIXELES_VE_o_RECUERDA, ), dtype=np.float32, name='observation')      
    # inicializa parámetros generales
    self._cantMaxIteraciones = cantMaxIteraciones
    # nota: tener en cuenta que si la cantidad de iteraciones es muy grande, 
    # se vuelve búsqueda exhaustiva y siempre queda muy cerca
    self._hashMapaBusq = Hash_configuracion_Mapa_Busqueda
    self._resetCambiaTodo = resetCambiaTodo  
    # inicializa variables 
    self._episode_ended = False
    self._state = 0
    # define configuración del entorno
    if self._resetCambiaTodo:
      # cada vez que se resetea se define
      self._mapaBusq = None      
    else:
      # sólo se generan al iniciar y se mantiene la misma
      self._mapaBusq = self.crearEspacioBusqueda(self._hashMapaBusq)    
    self._antTurtle = None
    self._cantIteraciones = 0

  def action_spec(self):
    # devuelve la forma de las acciones
    return self._action_spec

  def observation_spec(self):
    # devuelve la forma de las observaciones   
    return self._observation_spec

  def _reset(self):
    # resetea el entorno
    # crea partículas
    if self._resetCambiaTodo:
      # cada vez que se reseta, se define la lista de partículas
      self._mapaBusq = self.crearEspacioBusqueda(self._hashMapaBusq)    
    # resetea la hormiga-tortuga
    self._antTurtle = HormigaTortuga(self._mapaBusq.posHormiguero)
    # actualiza el estado considerando cantidad de ordenados
    self._state = 0
    self._cantIteraciones = 0
    self._episode_ended = False
    return ts.restart(self.devolverObsActual())

  def crearEspacioBusqueda(self, Hash_configuracion_Mapa_Busqueda=None):
    if Hash_configuracion_Mapa_Busqueda is None or Hash_configuracion_Mapa_Busqueda=="":
      # crea el espacio de búsqueda nuevo
      mapa = MapaBusqueda(limMin=MIN_ESPACIO_BUSQ, limMax=MAX_ESPACIO_BUSQ, \
                          cantMaximosLocales=CANT_MAXIMOS_LOCALES, cantObstaculos=CANT_OBSTACULOS)
    else:
      # usa configuración de mapa de búsqueda definida en hash
      mapa = pickle.loads(codecs.decode(Hash_configuracion_Mapa_Busqueda.encode(), "base64"))
    # devuelve el mapa y el valor mínimo para finalizar la búsqueda
    return mapa

  def devolverObsActual(self):
    # devuelve valores para la observación actual
    obs = []
    # lo que VE actualmente
    # devuelve 8 listas:
    #       5 listas (adelante y costados) con valores:
    #         -1 para obstáculos o limites
    #          0 para posiciones normales
    #         >0 para máximos
    #       3 listas (atrás) con valores en 0
    obs.append(  self._antTurtle.ve(self._mapaBusq, OBS_MAX_PIXELES_VE_o_RECUERDA)  )
    # lo que RECUERDA de ciclos anteriores
    # devuelve 8 listas con valores:
    #                        1 para posiciones que no paso    
    #                        0 para posiciones por las que paso
    obs.append(  self._antTurtle.recuerda(self._mapaBusq, OBS_MAX_PIXELES_VE_o_RECUERDA)  )
    ##print(np.array(obs).shape)
    # formatea la salida como array float
    # nota: para DQN parece ser que conviene 
    # normalizar los valores para que sean más homogeneos 
    # y no demasiado dispares entre sí 
    # (sino genera un 'loss' demasiado grande)        
    res = []
    for listaTipo in obs:
      auxT = []
      for listaVal in listaTipo:
        auxV = []
        for val in listaVal:
          auxV.append( round(val,4) )
        auxT.append( np.array(auxV, dtype=np.float32) )
      res.append( np.array(auxT, dtype=np.float32) )
    ##print(len(res), "->", res)
    res = np.array(res, dtype=np.float32) 
    ##print(res.shape)    
    return res

  def _step(self, action):
    # aplica una acción sobre el entorno
    
    if self._episode_ended:
      # si el entorno está finalizado, lo resetea
      return self.reset()

    # actualiza cantidad de interacciones 
    self._cantIteraciones = self._cantIteraciones - 1

    # parsea la accion para determinar acción
    idAccion, param_grados, param_pasos = parsearAccion(action)

    # aplica la acción correspondiente 
    self._state = self._antTurtle.desplazarse(self._mapaBusq, param_grados, param_pasos)
    self._state = self._state / 10

    # determina si debe finalizar o no
    terAlcanzaMaximo = self._antTurtle.recorridoEncuentraSolucion
    terLlegoMaxIteraciones = (abs(self._cantIteraciones) >= abs(self._cantMaxIteraciones))
    if terAlcanzaMaximo or terLlegoMaxIteraciones:
      # si lllegó a algún máximp
      # o si la cantidad de iteraciones llega al límite
      # fuerza que finaliza
      self._episode_ended = True

    if self._episode_ended:
      # si finaliza
      # devuelve el reward final (siempre se maximiza)
      # usando la heuristica de la solución hallada
      if terAlcanzaMaximo:
        r = self._antTurtle.valRecorridoUltimo
      else:
        r = 0
      return ts.termination(self.devolverObsActual(), reward=round(r, 4))
    else:
      # si no finaliza
      return ts.transition(
         self.devolverObsActual(), reward=round(self._state, 4), discount=0.9)
         # notar que no se usa discount=1.0 porque sino genera problema de 'loss' muy grande

  def render(self, mode = 'human'):
    # muestra información sobre el entorno
    if  self._cantIteraciones==0:
      # Muestra la información sobre el Mapa de Búsqueda
      print("\n> Mapa de Búsqueda:")
      # devuele el hash del mapa de búsqueda
      print("    * Hash del mapa de búsqueda definido: ")
      print(codecs.encode(pickle.dumps(self._mapaBusq), "base64").decode() )
      print("    ++ Ubicación del Hormiguero: ", self._mapaBusq.posHormiguero)
      strMax = ""
      for auxPos, auxVal in zip(self._mapaBusq.posMaximos, self._mapaBusq.valMaximos):
          if strMax != "":
            strMax = strMax  + ", "
          strMax = strMax + str(auxPos) + " { " + str(auxVal) + " } "
      print("    ** Máximos Generados: ", len(self._mapaBusq.posMaximos), " --> ", strMax)
      print("    ** Posición Máximo Óptimo: ", self._mapaBusq.posMaximoGlobal, " { ",  self._mapaBusq.valMaximoGlobal," } **")
      print("\n")  
    # muestra el valor del recorrido
    print("> Valor del Recorrido : ", self._antTurtle.valRecorridoUltimo)
    print("    ", self._antTurtle.recorridoUltimo)
    print("\n")    
    res = self._antTurtle.valRecorridoUltimo
    if self._episode_ended:
      # si termino genera el gráfico con el video de la animación
      ani = PrepararGrafico(self._mapaBusq, MIN_ESPACIO_BUSQ, MAX_ESPACIO_BUSQ, self._antTurtle.recorridoUltimo)    
      # Nota: esto se agega para que funcione en Google Colab
      rc('animation', html='jshtml')
      display(ani)
    return np.array(res, dtype=np.float32)

print("\nEntorno del Problema definido.")


In [ ]:
#@title Definir Simulador del Entorno y Entornos para Entrenamiento 

# si se indica este  hash del mapa de búsqueda 
# siempre usa esta configuración para entrenamiento del agente
Max_Iteraciones_Entorno_para_Entrenamiento =  250#@param {type:"integer"}
if Max_Iteraciones_Entorno_para_Entrenamiento < 1:
  Max_Iteraciones_Entorno_para_Entrenamiento = 10
Usar_hash_configuracion_fija_para_Entrenamiento = False #@param {type:"boolean"}
Hash_configuracion_Mapa_Busqueda_Entrenamiento = "gANjX19tYWluX18KTWFwYUJ1c3F1ZWRhCnEAKYFxAX1xAihYBgAAAGxpbU1pbnEDSwBYBgAAAGxp bU1heHEESx5YBwAAAGNhbnRNYXhxBUsBWA4AAABjYW50T2JzdGFjdWxvc3EGS2RYCgAAAEhvcm1p Z3Vlcm9xB11xCChLE0sRZVgKAAAATWF4aW1vc1Bvc3EJXXEKXXELKEsSSxZlYVgKAAAATWF4aW1v c1ZhbHEMXXENS2RhWAsAAABNYXhPcHRpbW9JRHEOSwBYDQAAAE9ic3RhY3Vsb3NQb3NxD11xEChd cREoSxVLEmVdcRIoSw1LFmVdcRMoSxpLDmVdcRQoSwpLFmVdcRUoSwlLHmVdcRYoSwdLA2VdcRco SwZLB2VdcRgoSwRLFWVdcRkoSxRLAmVdcRooSxBLC2VdcRsoSwRLB2VdcRwoSxNLGWVdcR0oSxVL EWVdcR4oSwFLEmVdcR8oSw1LFGVdcSAoSwpLGGVdcSEoSxdLFmVdcSIoSw1LG2VdcSMoSw9LB2Vd cSQoSxBLBWVdcSUoSw5LFWVdcSYoSwFLDGVdcScoSw9LD2VdcSgoSxFLG2VdcSkoSxlLE2VdcSoo Sx5LF2VdcSsoSwVLCGVdcSwoSxJLD2VdcS0oSwdLBGVdcS4oSwRLHGVdcS8oSwRLDWVdcTAoSwxL F2VdcTEoSx1LHmVdcTIoSxhLF2VdcTMoSwNLE2VdcTQoSwdLGmVdcTUoSxhLCmVdcTYoSwVLAWVd cTcoSxJLAmVdcTgoSwlLAmVdcTkoSwhLEWVdcTooSx5LAGVdcTsoSwdLAWVdcTwoSwlLBmVdcT0o SwdLEWVdcT4oSwdLCWVdcT8oSw5LGmVdcUAoSxZLEmVdcUEoSxFLBWVdcUIoSxxLEmVdcUMoSwBL EGVdcUQoSx1LC2VdcUUoSxFLDGVdcUYoSwJLEmVdcUcoSxlLF2VdcUgoSwFLDGVdcUkoSx1LD2Vd cUooSxlLD2VdcUsoSw1LHGVdcUwoSwNLAWVdcU0oSxxLDGVdcU4oSw1LB2VdcU8oSxZLDmVdcVAo SwxLFGVdcVEoSwlLFmVdcVIoSwFLEGVdcVMoSx5LAGVdcVQoSwZLEmVdcVUoSwRLGWVdcVYoSx5L HmVdcVcoSxxLCWVdcVgoSxFLCGVdcVkoSxZLAmVdcVooSxJLFGVdcVsoSxBLFmVdcVwoSwtLEGVd cV0oSwJLHmVdcV4oSxxLHGVdcV8oSwVLEmVdcWAoSwNLBmVdcWEoSxVLGWVdcWIoSwZLEGVdcWMo SxdLDWVdcWQoSxNLC2VdcWUoSwNLGWVdcWYoSw9LHGVdcWcoSwxLAWVdcWgoSxRLCWVdcWkoSw9L G2VdcWooSxhLF2VdcWsoSw5LD2VdcWwoSwNLD2VdcW0oSwdLD2VdcW4oSxpLHmVdcW8oSwxLF2Vd cXAoSxpLCmVdcXEoSwZLDmVdcXIoSwtLAGVdcXMoSxlLDWVdcXQoSwpLC2VlWA0AAABtYXBhRmVy b21vbmFzcXV9cXZYBwAAAGZlcm9EZnRxd0sBdWIu" #@param {type:"string"}
if Usar_hash_configuracion_fija_para_Entrenamiento:
  hash = Hash_configuracion_Mapa_Busqueda_Entrenamiento
else:
  hash = None

# Definir entornos de entrenamiento y de evaluación
# (ambos con lista que se cambia cada vez que se resetea)
# ya definidos dentro del wrapper para convertir en entornos TF
train_env = tf_py_environment.TFPyEnvironment( AntTurtleBuscarMaxMapaEntorno(Max_Iteraciones_Entorno_para_Entrenamiento, True, hash) )
eval_env = tf_py_environment.TFPyEnvironment( AntTurtleBuscarMaxMapaEntorno(Max_Iteraciones_Entorno_para_Entrenamiento, True, hash) )

# define política al azar independiente del Agente
random_policy = random_tf_policy.RandomTFPolicy(train_env.time_step_spec(),
                                                train_env.action_spec())

print("\nEntornos de entrenamiento y prueba definidos. ")

# definir simulador para probar el entorno
def SimularEntorno(env, policy, titulo, mostrarDetalle=True):
    print("\n** ", titulo, "**")                   
    # muesta estado inicial
    time_step = env.reset()      
    # muestra la información del entorno
    env.pyenv.render()
    if mostrarDetalle:
      print(" Ini: [", time_step, "]")    
    j = 1
    strAccRew = " "
    while not time_step.is_last():
      # la política determina la acción a realizar
      action_step = policy.action(time_step)
      time_step = env.step(action_step.action)
      # recupera la observación y muestra el nuevo estado 
      ac = action_step.action.numpy()[0]
      idAccion, grados, pasos = parsearAccion(ac)
      r = time_step.reward.numpy()[0]
      #ob = time_step.observation.numpy()[0]
      descAccion = POSIBLES_ACCIONES_DESC[ idAccion ] + "(" + str(grados) + "°, " + str(pasos) + "p)" 
      if mostrarDetalle:
        print("  #", j, ": acción ", descAccion, "-> Estado/Reward ", r, "[", time_step, ",", action_step, "]")
      else:
        if j > 1:
          strAccRew = strAccRew  + " + "
        strAccRew = strAccRew + descAccion + " {" + "{:.4f}".format(r) + "}" 
        if (j % 3) == 0:
          strAccRew = strAccRew  + "\n "
      j = j + 1
    # muestra estado final
    print( "> " + str(j-1)  + " Acciones: \n", strAccRew )
    # devuelve el video que genera
    resRender = env.pyenv.render()
    return resRender[0]

print("\nSimulador del entorno definido.")

# Probar el entorno definido con Política Aleatoria (opcional)
Probar_Entorno = "SI sin Detalle" #@param ["SI con Detalle", "SI sin Detalle", "NO"]
Probar_Entorno_Bool = (Probar_Entorno != "NO")
Mostrar_Detalle_Probar_Entorno_Bool = (Probar_Entorno == "SI con Detalle")
if Probar_Entorno_Bool:
   SimularEntorno(eval_env, random_policy, "Probando el entorno del problema con política al azar", Mostrar_Detalle_Probar_Entorno_Bool)
 

2) Establecer clase para el Agente:

In [ ]:
#@title Definir el Agente (tipo DQN o DQN Categórico)

tipo_agente = "DQN" #@param ["DQN", "DQN Categorico (C51)"]
learning_rate = 1e-3  # @param {type:"number"}
cant_neuronas_ocultas = "64, 16, 8" # @param {type:"string"}
DQN_usa_capas_convNet = True # @param {type:"boolean"}
DQNCat_num_atoms = 51  # @param {type:"integer"}

# controla cantidad de atoms para DQN Cat
if DQNCat_num_atoms <= 1:
  DQNCat_num_atoms = 51

# Define cantidad de neuronas ocultas para RNA-Q
hidden_layers = []
for val in cant_neuronas_ocultas.split(','):
  if  int(val) < 1:
    hidden_layers.append( 10 )
  else:
    hidden_layers.append( int(val) )
fc_layer_params = tuple(hidden_layers, )

if tipo_agente=="DQN":

  #define las capas convolutional
  if DQN_usa_capas_convNet:
    # nota: ojo que si se pone más de una capa convolutional, tira errore el entorno
    CNN_preprocessing_layers = tf.keras.models.Sequential(
                                        [tf.keras.layers.Conv2D(2, 2, activation='relu', padding="same"),
                                         tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
                                         tf.keras.layers.Flatten()])
  else:
    CNN_preprocessing_layers = None

  # Define RNA-Q
  q_net = q_network.QNetwork(
      train_env.observation_spec(),
      train_env.action_spec(),
      preprocessing_layers=CNN_preprocessing_layers,
      fc_layer_params=fc_layer_params)

  optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)

  train_step_counter = tf.Variable(0)

  # Define el agente de tipo Q
  ag = dqn_agent.DqnAgent(
      train_env.time_step_spec(),
      train_env.action_spec(),
      q_network=q_net,
      optimizer=optimizer,
      td_errors_loss_fn=common.element_wise_squared_loss,
      train_step_counter=train_step_counter)

  ag.initialize()

  print("Agente DQN inicializado. ")

elif tipo_agente == "DQN Categorico (C51)":
  
  # Define RNA-Q Categórico
  categorical_q_net = categorical_q_network.CategoricalQNetwork(
      train_env.observation_spec(),
      train_env.action_spec(),
      num_atoms=DQNCat_num_atoms,
      fc_layer_params=fc_layer_params)

  optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)

  train_step_counter = tf.compat.v2.Variable(0)
  
  # parámetros especificos (por defecto)
  n_step_update = 2
  gamma = 0.99

  # Define el agente de tipo Q Categórico
  ag = categorical_dqn_agent.CategoricalDqnAgent(
      train_env.time_step_spec(),
      train_env.action_spec(),
      categorical_q_network=categorical_q_net,
      optimizer=optimizer,
      min_q_value=0,
      max_q_value=MAXIMO_VALOR_ACTION,
      n_step_update=n_step_update,
      td_errors_loss_fn=common.element_wise_squared_loss,
      gamma=gamma,
      train_step_counter=train_step_counter)
  
  ag.initialize()
  
  print("Agente DQN Categorico (C51) inicializado. ")


3) Llevar a cabo el Entrenamiento:

In [ ]:
#@title Definir Métricas para evaluación

# Se usa el promedio de la recompensa (la más común)
# See also the metrics module for standard implementations of different metrics.
# https://github.com/tensorflow/agents/tree/master/tf_agents/metrics

def compute_avg_return(environment, policy, num_episodes=10):

  if num_episodes <= 0:
    return 0.0
    
  total_return = 0.0
  for _ in range(num_episodes):

    time_step = environment.reset()
    episode_return = 0.0
    while not time_step.is_last():
      action_step = policy.action(time_step)
      time_step = environment.step(action_step.action)
      episode_return += time_step.reward
    total_return += episode_return

  avg_return = total_return / num_episodes
  return avg_return.numpy()[0]

print("Métricas definidas.")

In [ ]:
#@title Preparar datos para Entrenamiento

initial_collect_steps =   10000# @param {type:"integer"} 
collect_steps_per_iteration = 100  # @param {type:"integer"}
replay_buffer_max_length = 100000  # @param {type:"integer"}
batch_size = 64  # @param {type:"integer"}

# Define 'Replay Buffer' para que el agente recuerde las observaciones realizadas
replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec = ag.collect_data_spec,
    batch_size = train_env.batch_size,
    max_length = replay_buffer_max_length)

# Recolecta datos generados al azar
# This loop is so common in RL, that we provide standard implementations. 
# For more details see the drivers module.
# https://www.tensorflow.org/agents/api_docs/python/tf_agents/drivers

def collect_step(environment, policy, buffer):
  time_step = environment.current_time_step()
  action_step = policy.action(time_step)
  next_time_step = environment.step(action_step.action)
  traj = trajectory.from_transition(time_step, action_step, next_time_step)

  # Add trajectory to the replay buffer
  buffer.add_batch(traj)

def collect_data(env, policy, buffer, steps=1):
  for _ in range(steps):
    collect_step(env, policy, buffer)

collect_data(train_env, random_policy, replay_buffer, initial_collect_steps)

print("\nDatos recolectados.")

# Muestra ejemplo de los datos recolectados
##iter(replay_buffer.as_dataset()).next()

if tipo_agente=="DQN":
  # Preparar los datos recolectados con trajectories de shape [Bx2x...]
  dataset = replay_buffer.as_dataset(
      num_parallel_calls=3, 
      sample_batch_size=batch_size, 
      num_steps=2).prefetch(3)
elif tipo_agente == "DQN Categorico (C51)":
  # Dataset generates trajectories with shape [BxTx...] where
  # T = n_step_update + 1.
  dataset = replay_buffer.as_dataset(
      num_parallel_calls=3, sample_batch_size=batch_size,
      num_steps=n_step_update + 1).prefetch(3)

iterator = iter(dataset)

# Muestra ejemplo 
##iterator.next()
print("\nDataset creado.")

In [ ]:
#@title Entrenar al Agente

cant_ciclos_entrenamiento_finalizar =  10000# @param {type:"integer"}
minima_recompensa_promedio_finalizar = 100 # @param {type:"integer"}
log_cada_ciclos = 200  # @param {type:"integer"}
mostar_recompensa_cada = 1000  # @param {type:"integer"}
cant_episodios_evaluacion =  10# @param {type:"integer"}

#  Optimize by wrapping some of the code in a graph using TF function (Optional)
ag.train = common.function(ag.train)

# Reset the train step
ag.train_step_counter.assign(0)

# Evaluate the agent's policy once before training.
avg_return = compute_avg_return(eval_env, ag.policy, cant_episodios_evaluacion)
ar_ciclo = []
ar_returns = []
ar_loss = []

print("\n** Comienza el Entrenamiento **\n")
for _ in range(cant_ciclos_entrenamiento_finalizar):

  # Collect a few steps using collect_policy and save to the replay buffer.
  collect_data(train_env, ag.collect_policy, replay_buffer, collect_steps_per_iteration)

  # Sample a batch of data from the buffer and update the agent's network.
  experience, unused_info = next(iterator)
  train_loss = ag.train(experience).loss

  step = ag.train_step_counter.numpy()

  if (step == 1) or (step == cant_ciclos_entrenamiento_finalizar) or (step % log_cada_ciclos == 0):
    print('step = {0}: loss = {1:.3f}'.format(step, train_loss))    
    ar_ciclo.append( step )
    ar_loss.append( train_loss )
    avg_return = compute_avg_return(eval_env, ag.policy, cant_episodios_evaluacion)
    ar_returns.append( avg_return )

    if (step == 1) or (step == cant_ciclos_entrenamiento_finalizar) or (step % mostar_recompensa_cada == 0):
      print('step = {0}: Promedio Recompensa = {1:.1f}'.format(step, avg_return))

    if (avg_return >= minima_recompensa_promedio_finalizar):
      print('** Finaliza en step {0} por buen valor de recompensa promedio: {1:.1f}'.format(step, avg_return)) 
      break

print("\n** Entrenamiento Finalizado **\n")


In [ ]:
#@title Mostrar Gráficos del Entrenamiento


plt.figure(figsize=(12,5)) 
plt.plot( ar_ciclo, ar_returns)
plt.title("Resultados del Entrenamiento del Agente - Promedio Recompensa")
#plt.legend(['Promedio Recompensa', 'Loss de Entrenamiento'], loc='upper right')
plt.ylabel('Valor')
plt.xlabel('Ciclo')
plt.xlim(right=max(ar_ciclo))   
plt.grid(True)
plt.show()

plt.figure(figsize=(12,5)) 
#plt.plot( ar_ciclo, ar_returns)
plt.plot( ar_ciclo, ar_loss, color="red" )
plt.title("Resultados del Entrenamiento del Agente - Loss de Entrenamiento")
#plt.legend(['Promedio Recompensa', 'Loss de Entrenamiento'], loc='upper right')
plt.ylabel('Valor')
plt.xlabel('Ciclo')
plt.xlim(right=max(ar_ciclo))   
plt.grid(True)
plt.show()


4) Cargar / Graba el modelo de las políticas entrenadas:

In [ ]:
#@title Cargar o Guardar el Modelo
# parámetros
directorio_modelo = '/content/gdrive/MyDrive/IA/demoAgentes/Modelos' #@param {type:"string"}
nombre_modelo_grabar = "policy-Ant-Trutle-RandomPosProblem" #@param {type:"string"}
accion_realizar = "-" #@param ["-", "Cargar Modelo", "Grabar Modelo"]

# determina lugar donde se guarda el modelo
policy_dir = os.path.join(directorio_modelo, nombre_modelo_grabar)

if accion_realizar != "-":
  # Montar Drive
  from google.colab import drive
  drive.mount('/content/gdrive')
if accion_realizar == "Grabar Modelo":
  # guarda la politica del agente entrenado
  tf_policy_saver = policy_saver.PolicySaver(ag.policy)
  tf_policy_saver.save(policy_dir)
  print("\nPolítica del modelo guardada en ", policy_dir)
elif accion_realizar == "Cargar Modelo":
  # carga la política del modelo
  saved_policy = tf.compat.v2.saved_model.load(policy_dir)
  print("\nPolítica del modelo recuperada de ", policy_dir)

5) Probar entrenamiento comparando resultados:

In [ ]:
#@title Realizar una prueba del Agente Entrenado contra el Azar
Max_Iteraciones_Entorno_para_probar =  250#@param {type:"integer"}
if Max_Iteraciones_Entorno_para_probar < 1:
  Max_Iteraciones_Entorno_para_probar = 1
Usar_hash_para_probar = False #@param {type:"boolean"}
Hash_configuracion_Mapa_Busqueda_probar = "gANjX19tYWluX18KTWFwYUJ1c3F1ZWRhCnEAKYFxAX1xAihYBgAAAGxpbU1pbnEDSwBYBgAAAGxp bU1heHEESx5YBwAAAGNhbnRNYXhxBUsBWA4AAABjYW50T2JzdGFjdWxvc3EGS2RYCgAAAEhvcm1p Z3Vlcm9xB11xCChLE0sRZVgKAAAATWF4aW1vc1Bvc3EJXXEKXXELKEsSSxZlYVgKAAAATWF4aW1v c1ZhbHEMXXENS2RhWAsAAABNYXhPcHRpbW9JRHEOSwBYDQAAAE9ic3RhY3Vsb3NQb3NxD11xEChd cREoSxVLEmVdcRIoSw1LFmVdcRMoSxpLDmVdcRQoSwpLFmVdcRUoSwlLHmVdcRYoSwdLA2VdcRco SwZLB2VdcRgoSwRLFWVdcRkoSxRLAmVdcRooSxBLC2VdcRsoSwRLB2VdcRwoSxNLGWVdcR0oSxVL EWVdcR4oSwFLEmVdcR8oSw1LFGVdcSAoSwpLGGVdcSEoSxdLFmVdcSIoSw1LG2VdcSMoSw9LB2Vd cSQoSxBLBWVdcSUoSw5LFWVdcSYoSwFLDGVdcScoSw9LD2VdcSgoSxFLG2VdcSkoSxlLE2VdcSoo Sx5LF2VdcSsoSwVLCGVdcSwoSxJLD2VdcS0oSwdLBGVdcS4oSwRLHGVdcS8oSwRLDWVdcTAoSwxL F2VdcTEoSx1LHmVdcTIoSxhLF2VdcTMoSwNLE2VdcTQoSwdLGmVdcTUoSxhLCmVdcTYoSwVLAWVd cTcoSxJLAmVdcTgoSwlLAmVdcTkoSwhLEWVdcTooSx5LAGVdcTsoSwdLAWVdcTwoSwlLBmVdcT0o SwdLEWVdcT4oSwdLCWVdcT8oSw5LGmVdcUAoSxZLEmVdcUEoSxFLBWVdcUIoSxxLEmVdcUMoSwBL EGVdcUQoSx1LC2VdcUUoSxFLDGVdcUYoSwJLEmVdcUcoSxlLF2VdcUgoSwFLDGVdcUkoSx1LD2Vd cUooSxlLD2VdcUsoSw1LHGVdcUwoSwNLAWVdcU0oSxxLDGVdcU4oSw1LB2VdcU8oSxZLDmVdcVAo SwxLFGVdcVEoSwlLFmVdcVIoSwFLEGVdcVMoSx5LAGVdcVQoSwZLEmVdcVUoSwRLGWVdcVYoSx5L HmVdcVcoSxxLCWVdcVgoSxFLCGVdcVkoSxZLAmVdcVooSxJLFGVdcVsoSxBLFmVdcVwoSwtLEGVd cV0oSwJLHmVdcV4oSxxLHGVdcV8oSwVLEmVdcWAoSwNLBmVdcWEoSxVLGWVdcWIoSwZLEGVdcWMo SxdLDWVdcWQoSxNLC2VdcWUoSwNLGWVdcWYoSw9LHGVdcWcoSwxLAWVdcWgoSxRLCWVdcWkoSw9L G2VdcWooSxhLF2VdcWsoSw5LD2VdcWwoSwNLD2VdcW0oSwdLD2VdcW4oSxpLHmVdcW8oSwxLF2Vd cXAoSxpLCmVdcXEoSwZLDmVdcXIoSwtLAGVdcXMoSxlLDWVdcXQoSwpLC2VlWA0AAABtYXBhRmVy b21vbmFzcXV9cXZYBwAAAGZlcm9EZnRxd0sBdWIu" #@param {type:"string"}
if Usar_hash_para_probar:
  hash = Hash_configuracion_Mapa_Busqueda_probar
else:
  hash = None

# variable auxiliares
cantidad_probar =  1
promAzar = 0
promAgente = 0

# determina política a usar
policy_agente_entrenado = None
if not('ag' in vars() or 'ag' in globals()) or ag is None:
  if not('saved_policy' in vars() or 'saved_policy' in globals()) or saved_policy is None:
    ValueError("No hay política entrenada definida.")
  else:
    policy_agente_entrenado = saved_policy
    print("- Se usa la política recuperada del drive.")
else:
  policy_agente_entrenado = ag.policy
  print("- Se usa la política del modelo entrenado.")

for i in range(cantidad_probar):

  if cantidad_probar > 1:
    print("\n> Prueba ", i+1, ":")

  # crea nuevo entorno que mantiene el espacio de búsqueda
  prueba_env =  tf_py_environment.TFPyEnvironment( AntTurtleBuscarMaxMapaEntorno(Max_Iteraciones_Entorno_para_probar, False, hash) )

  # Probar Aleatorio
  valorAzar = SimularEntorno(prueba_env, random_policy, "Resultados Aleatorio", False) 
  promAzar = promAzar + valorAzar
  
  print("\n ************************************************************************************************************")
  
  # Probar Agente Entrenado
  valorAgente = SimularEntorno(prueba_env, policy_agente_entrenado, "Resultados de Agente Entrenado", False) 
  promAgente = promAgente + valorAgente

  # Decide Ganador
  if valorAzar < valorAgente:
    print("\n--> El Agente Entrenado (", valorAgente,") genera MEJOR resultado que el azar (", valorAzar,")")
  else:
    print("\n--> El Agente Entrenado (", valorAgente,") genera PEOR resultado que el azar (", valorAzar,")")

# Decide Ganador General
if cantidad_probar > 1:
  promAgente = promAgente / cantidad_probar
  promAzar = promAzar / cantidad_probar
  print("\n================================================================================================\n")
  if promAzar < promAgente:
    print("= En promedio, el Agente Entrenado (", promAgente,") tiene MEJORES resultado que  el azar (", promAzar,")")
  else:
    print("= En promedio, el Agente Entrenado (", promAgente,") tiene PEORES resultados que el azar (", promAzar,")")
  print("\n================================================================================================\n")

In [ ]:
#@title Realizar varias pruebas del Agente Entrenado contra el Azar 
Max_Iteraciones_Entorno_para_probar = 250 #@param {type:"integer"}
if Max_Iteraciones_Entorno_para_probar < 1:
  Max_Iteraciones_Entorno_para_probar = 1
Usar_hash_para_probar = False #@param {type:"boolean"}
Hash_configuracion_Mapa_Busqueda_probar = "gANjX19tYWluX18KTWFwYUJ1c3F1ZWRhCnEAKYFxAX1xAihYBgAAAGxpbU1pbnEDSwBYBgAAAGxp bU1heHEESx5YBwAAAGNhbnRNYXhxBUsBWA4AAABjYW50T2JzdGFjdWxvc3EGS2RYCgAAAEhvcm1p Z3Vlcm9xB11xCChLE0sRZVgKAAAATWF4aW1vc1Bvc3EJXXEKXXELKEsSSxZlYVgKAAAATWF4aW1v c1ZhbHEMXXENS2RhWAsAAABNYXhPcHRpbW9JRHEOSwBYDQAAAE9ic3RhY3Vsb3NQb3NxD11xEChd cREoSxVLEmVdcRIoSw1LFmVdcRMoSxpLDmVdcRQoSwpLFmVdcRUoSwlLHmVdcRYoSwdLA2VdcRco SwZLB2VdcRgoSwRLFWVdcRkoSxRLAmVdcRooSxBLC2VdcRsoSwRLB2VdcRwoSxNLGWVdcR0oSxVL EWVdcR4oSwFLEmVdcR8oSw1LFGVdcSAoSwpLGGVdcSEoSxdLFmVdcSIoSw1LG2VdcSMoSw9LB2Vd cSQoSxBLBWVdcSUoSw5LFWVdcSYoSwFLDGVdcScoSw9LD2VdcSgoSxFLG2VdcSkoSxlLE2VdcSoo Sx5LF2VdcSsoSwVLCGVdcSwoSxJLD2VdcS0oSwdLBGVdcS4oSwRLHGVdcS8oSwRLDWVdcTAoSwxL F2VdcTEoSx1LHmVdcTIoSxhLF2VdcTMoSwNLE2VdcTQoSwdLGmVdcTUoSxhLCmVdcTYoSwVLAWVd cTcoSxJLAmVdcTgoSwlLAmVdcTkoSwhLEWVdcTooSx5LAGVdcTsoSwdLAWVdcTwoSwlLBmVdcT0o SwdLEWVdcT4oSwdLCWVdcT8oSw5LGmVdcUAoSxZLEmVdcUEoSxFLBWVdcUIoSxxLEmVdcUMoSwBL EGVdcUQoSx1LC2VdcUUoSxFLDGVdcUYoSwJLEmVdcUcoSxlLF2VdcUgoSwFLDGVdcUkoSx1LD2Vd cUooSxlLD2VdcUsoSw1LHGVdcUwoSwNLAWVdcU0oSxxLDGVdcU4oSw1LB2VdcU8oSxZLDmVdcVAo SwxLFGVdcVEoSwlLFmVdcVIoSwFLEGVdcVMoSx5LAGVdcVQoSwZLEmVdcVUoSwRLGWVdcVYoSx5L HmVdcVcoSxxLCWVdcVgoSxFLCGVdcVkoSxZLAmVdcVooSxJLFGVdcVsoSxBLFmVdcVwoSwtLEGVd cV0oSwJLHmVdcV4oSxxLHGVdcV8oSwVLEmVdcWAoSwNLBmVdcWEoSxVLGWVdcWIoSwZLEGVdcWMo SxdLDWVdcWQoSxNLC2VdcWUoSwNLGWVdcWYoSw9LHGVdcWcoSwxLAWVdcWgoSxRLCWVdcWkoSw9L G2VdcWooSxhLF2VdcWsoSw5LD2VdcWwoSwNLD2VdcW0oSwdLD2VdcW4oSxpLHmVdcW8oSwxLF2Vd cXAoSxpLCmVdcXEoSwZLDmVdcXIoSwtLAGVdcXMoSxlLDWVdcXQoSwpLC2VlWA0AAABtYXBhRmVy b21vbmFzcXV9cXZYBwAAAGZlcm9EZnRxd0sBdWIu" #@param {type:"string"}
if Usar_hash_para_probar:
  hash = Hash_configuracion_Mapa_Busqueda_probar
else:
  hash = None

# variable auxiliares
cantidad_probar =  100 #@param {type:"integer"}

# determina política a usar
policy_agente_entrenado = None
if not('ag' in vars() or 'ag' in globals()) or ag is None:
  if not('saved_policy' in vars() or 'saved_policy' in globals()) or saved_policy is None:
    ValueError("No hay política entrenada definida.")
  else:
    policy_agente_entrenado = saved_policy
    print("- Se usa la política recuperada del drive.")
else:
  policy_agente_entrenado = ag.policy
  print("- Se usa la política del modelo entrenado.")


# crea nuevo entorno que mantiene el espacio de búsqueda
prueba_env =  tf_py_environment.TFPyEnvironment( AntTurtleBuscarMaxMapaEntorno(Max_Iteraciones_Entorno_para_probar, False, hash) )

# calcula promedio de política al azar
promAzar = compute_avg_return(prueba_env, random_policy, num_episodes=cantidad_probar)

# calcula promedio de política del agente entrenado
promAgente = compute_avg_return(prueba_env, policy_agente_entrenado, num_episodes=cantidad_probar)

# Decide Ganador General
#promAgente = round(promAgente / cantidad_probar, 3)
#promAzar = round(promAzar / cantidad_probar, 3)
print("\n================================================================================================\n")
if promAzar < promAgente:
  print("= En promedio, el Agente Entrenado (", promAgente,") tiene MEJORES resultado que  el azar (", promAzar,")")
else:
  print("= En promedio, el Agente Entrenado (", promAgente,") tiene PEORES resultados que el azar (", promAzar,")")
print("\n================================================================================================\n")